In [ ]:
# Ch 9-2 - Computing molecular distances on a PDB file

In [ ]:
# Load model
from Bio import PDB 
repository = PDB.PDBList() 
parser = PDB.PDBParser() 
repository.retrieve_pdb_file('1TUP', pdir='.', file_format='pdb') 
p53_1tup = parser.get_structure('P 53', 'pdb1tup.ent') 

In [ ]:
# Get Zincs
zns = []
for atom in p53_1tup.get_atoms(): 
    if atom.element == 'ZN': 
        zns.append(atom) 
for zn in zns: 
    print(zn, zn.coord) 

In [ ]:
# Get the distance between one atom and another set of atoms
import math 
def get_closest_atoms(pdb_struct, ref_atom, distance): 
    atoms = {} 
    rx, ry, rz = ref_atom.coord 
    for atom in pdb_struct.get_atoms(): 
        if atom == ref_atom: 
            continue 
        x, y, z = atom.coord 
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance: 
            atoms[atom] = my_dist 
    return atoms 

In [ ]:
# Compute atoms near zincs
for zn in zns: 
    print() 
    print(zn.coord) 
    atoms = get_closest_atoms(p53_1tup, zn, 4) 
    for atom, distance in atoms.items(): 
        print(atom.element, distance, atom.coord) 

In [ ]:
# How many atoms do we get as we increase the distance
for distance in [1, 2, 4, 8, 16, 32, 64, 128]: 
    my_atoms = [] 
    for zn in zns: 
        atoms = get_closest_atoms(p53_1tup, zn, distance) 
        my_atoms.append(len(atoms)) 
    print(distance, my_atoms) 

In [ ]:
# Time the output
import timeit 
nexecs = 10 
print(timeit.timeit('get_closest_atoms(p53_1tup, zns[0], 4.0)', 
    'from __main__ import get_closest_atoms, p53_1tup, zns', 
    number=nexecs) / nexecs * 1000) 

In [ ]:
# A better distance function
def get_closest_alternative(pdb_struct, ref_atom, distance): 
    atoms = {} 
    rx, ry, rz = ref_atom.coord 
    for atom in pdb_struct.get_atoms(): 
        if atom == ref_atom: 
            continue 
        x, y, z = atom.coord 
        if abs(x - rx) > distance or abs(y - ry) > distance or abs(z - rz) > distance: 
            continue 
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance: 
            atoms[atom] = my_dist 
    return atoms 

In [ ]:
# Now let's time it:
print(timeit.timeit('get_closest_alternative(p53_1tup, zns[0], 4.0)', 
    'from __main__ import get_closest_alternative, p53_1tup, zns', 
    number=nexecs) / nexecs * 1000) 

In [ ]:
# Compare the function with different distances
print('Standard') 
for distance in [1, 4, 16, 64, 128]: 
    print(timeit.timeit('get_closest_atoms(p53_1tup, zns[0], distance)', 
    'from __main__ import get_closest_atoms, p53_1tup, zns, distance', 
    number=nexecs) / nexecs * 1000) 
print('Optimized') 
for distance in [1, 4, 16, 64, 128]: 
    print(timeit.timeit('get_closest_alternative(p53_1tup, zns[0], distance)', 
        'from __main__ import get_closest_alternative, p53_1tup, zns, distance', 
        number=nexecs) / nexecs * 1000)

In [ ]:
## End of Notebook ##